# Great Energy Predictor - Featurization
#### Hosted by: ASHRAE
##### Source: https://www.kaggle.com/c/ashrae-energy-prediction

## Section I: Dependencies and Data

### Dependencies

In [1]:
# Dependencies
import os
import gc
import holidays
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from feature_engine.encoding import RareLabelEncoder, MeanEncoder, OrdinalEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression, RFE, SelectFromModel, SelectKBest, SelectPercentile
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import src.utils as udf

%matplotlib inline

In [2]:
# Dataframe settings
pd.set_option('display.max_columns', 50)

# Plot settings
mpl.style.use('seaborn')
mpl.rcParams['figure.figsize'] = (15, 3)
mpl.rcParams['font.size'] = 12

### Data

In [3]:
# Data path
data_path = os.path.join('..', 'data', '02-prep-out')
data_path

'../data/02-prep-out'

In [4]:
# Meter data
meter = pd.read_pickle(os.path.join(data_path, 'meter.pkl'))
meter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18205342 entries, 72 to 20216099
Data columns (total 11 columns):
 #   Column         Dtype         
---  ------         -----         
 0   building_id    uint16        
 1   meter          uint8         
 2   timestamp      datetime64[ns]
 3   meter_reading  float32       
 4   site_id        uint8         
 5   meter_type     object        
 6   dayofyear      uint16        
 7   month          uint8         
 8   day            uint8         
 9   dayofweek      uint8         
 10  hour           uint8         
dtypes: datetime64[ns](1), float32(1), object(1), uint16(2), uint8(6)
memory usage: 659.8+ MB


In [5]:
# Weather data
weather = pd.read_pickle(os.path.join(data_path, 'weather.pkl'))
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140544 entries, 0 to 140543
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   site_id             140544 non-null  uint8         
 1   timestamp           140544 non-null  datetime64[ns]
 2   air_temperature     140544 non-null  float32       
 3   dew_temperature     140544 non-null  float32       
 4   sea_level_pressure  140544 non-null  float32       
 5   wind_direction      140544 non-null  uint16        
 6   wind_speed          140544 non-null  float32       
dtypes: datetime64[ns](1), float32(4), uint16(1), uint8(1)
memory usage: 3.6 MB


In [6]:
# Building data
building = pd.read_pickle(os.path.join(data_path, 'building.pkl'))
building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   site_id       1449 non-null   uint8 
 1   building_id   1449 non-null   uint16
 2   primary_use   1449 non-null   object
 3   square_feet   1449 non-null   uint32
 4   year_built    1449 non-null   uint16
 5   missing_year  1449 non-null   uint8 
dtypes: object(1), uint16(2), uint32(1), uint8(2)
memory usage: 25.6+ KB


In [7]:
# Clean vars
del data_path
gc.collect()

48

## Section II: Feature Engineering

### Convert `wind_direction` to Cartesian components

In the weather data, the `wind_direction` was recorded as a compass direction, which is in the polar coordinate system. This won't work in the context of machine learning because the numeric values imply that this feature has an ordinal nature, and that's simply not true. A direction of 300 degrees is not more than a direction of 100 degrees; it's just in a different direction.

In order for machine learning models to be able to interpret `wind_direction` correctly, it needs to be using Cartesian coordinates. We will make this conversion by extracting the x and y components of the direction values, which together would describe the `wind_direction`.

Note: `wind_direction` contains both values of 0 and 360, which are the same direction. But in this case, a `wind_direction` of 0 just means that there was no wind, as evident by the 0 `wind_speed`.

In [8]:
# Extract x and y components from wind direction
weather = udf.polar_to_cartesian(weather, 'wind_direction')
weather.head(3)

,site_id,timestamp,air_temperature,dew_temperature,sea_level_pressure,wind_speed,wind_direction_x,wind_direction_y
0,0,2016-01-01 00:00:00,19.4,19.4,1019.400024,0.0,0.000000,0.0
1,0,2016-01-01 01:00:00,21.1,21.1,1019.400024,0.0,0.000000,0.0
2,0,2016-01-01 02:00:00,21.1,21.1,1018.799988,1.5,-0.866025,-0.5


### Create a new feature for relative humidity

The weather data contains both `air_temperature` and `dew_temperature`, which are highly correlated. Relative humidity is a weather feature that can be calculated using both `air_temperature` and `dew_temperature`. Relative humidity may effectively replace both by incorporating the information from both features and eliminating the multicollinearity. The calculation for this new feature follows the source below.

We won't be dropping `air_temperature` and `dew_temperature` just yet because we don't actually know which feature will be the most useful in building machine learning models until actually building them.

Source: https://www.weather.gov/media/epz/wxcalc/vaporPressure.pdf

In [9]:
# Add feat for relative humidity
weather['rel_humidity'] = udf.calc_rel_humidity(weather['air_temperature'], weather['dew_temperature'])
weather.describe()

,site_id,air_temperature,dew_temperature,sea_level_pressure,wind_speed,wind_direction_x,wind_direction_y,rel_humidity
count,140544.000000,140544.000000,140544.000000,140544.000000,140544.000000,140544.000000,1.405440e+05,140544.000000
mean,7.500000,14.356584,7.297376,1016.131287,3.562700,-0.037601,-9.005360e-02,67.503700
std,4.609789,10.669369,9.826284,7.883689,2.334334,0.663424,6.739621e-01,21.754589
min,0.000000,-28.900000,-35.000000,968.200012,0.000000,-1.000000,-1.000000e+00,3.751658
25%,3.750000,7.200000,0.600000,1011.599976,2.100000,-0.642788,-7.660444e-01,52.516804
50%,7.500000,15.000000,8.200000,1016.400024,3.100000,0.000000,-2.449294e-16,71.740288
75%,11.250000,22.200001,14.400000,1020.900024,5.000000,0.500000,5.000000e-01,85.883133
max,15.000000,47.200001,26.100000,1045.500000,19.000000,1.000000,1.000000e+00,119.269371


### Add a boolean indicator for weekends

As seen in the EDA, there was a bit of difference in energy usage between weekdays and weekends for some buildings, especially with electricity meters. It would be useful to have a boolean feature that indicates whether the meter reading was during the weekend. We will create this weekend indicator by checking if the `dayofweek` is 5 (Saturday) or 6 (Sunday).

In [10]:
# Create weekend feat
meter['is_weekend'] = meter['dayofweek'].apply(lambda d: int(d in [5, 6])).astype('uint8')
meter.tail(3)

,building_id,meter,timestamp,meter_reading,site_id,meter_type,dayofyear,month,day,dayofweek,hour,is_weekend
20216096,1445,0,2016-12-31 23:00:00,4.825000,15,electricity,366,12,31,5,23,1
20216098,1447,0,2016-12-31 23:00:00,159.574997,15,electricity,366,12,31,5,23,1
20216099,1448,0,2016-12-31 23:00:00,2.850000,15,electricity,366,12,31,5,23,1


### Add a boolean indicator for holidays

Just as with weekends, energy usage is likely different on holidays so we will create a boolean feature that indicates whether the meter reading was during a holiday. In order to do this, we must first label each site with its respective country in order to correctly identify which holidays to use.

In [11]:
# Add country feat
site_countries = {0: 'US', 1: 'UK', 2: 'US', 3: 'US', 4: 'US', 5: 'UK', 6: 'US', 7: 'CA', 
                  8: 'US', 9: 'US', 10: 'US', 11: 'CA', 12: 'IE', 13: 'US', 14: 'US', 15: 'US'}
meter['country'] = meter.site_id.map(site_countries)
meter.head(3)

,building_id,meter,timestamp,meter_reading,site_id,meter_type,dayofyear,month,day,dayofweek,hour,is_weekend,country
72,74,0,2016-01-01,12.603682,0,electricity,1,1,1,4,0,0,US
91,93,0,2016-01-01,15.364478,0,electricity,1,1,1,4,0,0,US
103,105,0,2016-01-01,23.303600,1,electricity,1,1,1,4,0,0,UK


In [12]:
# Holiday dates for each country
US_holidays = holidays.UnitedStates()     # sites 0, 2, 3, 4, 6, 8, 9, 10, 13, 14, 15 
CA_holidays = holidays.Canada()           # sites 7, 11
UK_holidays = holidays.UnitedKingdom()    # sites 1, 5
IE_holidays = holidays.Ireland()          # site 12

# Initialize holiday year
print(US_holidays['2016-01-01'])
print(CA_holidays['2016-01-01'])
print(UK_holidays['2016-01-01'])
print(IE_holidays['2016-01-01'])

New Year's Day
New Year's Day
New Year's Day
New Year's Day


In [13]:
# Convert holiday lists to dfs
US_holiday_df = pd.DataFrame(zip(US_holidays.keys(), ['US'] * len(US_holidays), [1] * len(US_holidays)), 
                              columns=['date', 'country', 'is_holiday'])
CA_holiday_df = pd.DataFrame(zip(CA_holidays.keys(), ['CA'] * len(CA_holidays), [1] * len(CA_holidays)), 
                              columns=['date', 'country', 'is_holiday'])
UK_holiday_df = pd.DataFrame(zip(UK_holidays.keys(), ['UK'] * len(UK_holidays), [1] * len(UK_holidays)), 
                              columns=['date', 'country', 'is_holiday'])
IE_holiday_df = pd.DataFrame(zip(IE_holidays.keys(), ['IE'] * len(IE_holidays), [1] * len(IE_holidays)), 
                              columns=['date', 'country', 'is_holiday'])

# Concat holiday dfs
holiday_df = pd.concat([US_holiday_df, CA_holiday_df, UK_holiday_df, IE_holiday_df]).reset_index(drop=True)
holiday_df.loc[[0, 15, 30, 45]]

,date,country,is_holiday
0,2016-01-01,US,1
15,2016-07-01,CA,1
30,2016-07-12,UK,1
45,2016-12-27,IE,1


In [ ]:
# Sync date cols in both dfs
meter['date'] = meter['timestamp'].dt.date

# Merge holiday feat into meter df
meter = pd.merge(meter, holiday_df, on=['date', 'country'], how='left')
meter['is_holiday'] = meter['is_holiday'].fillna(0).astype('int8')
meter.head(3)

In [ ]:
# # Create holiday feat
# meter['is_holiday'] = meter.apply(lambda r: r['timestamp'] in eval(r['country'] + '_holidays'), axis=1).astype('uint8')
# meter.head(3)

### Group UK and IE into 1 label: EU

Now that the `country` has been used to create the `is_holiday` feature, we can update it in a way that minimizes rare labels. Only 1 site is in Ireland, so Ireland will be grouped with the United Kingdom to make "EU".

In [ ]:
# Group "UK" and "IE" into "EU" in country
meter['country'] = meter['country'].str.replace(r'UK|IE', 'EU')
meter.country.value_counts()

### Drop `day` and `date`

The `day` feature is being dropped because it will no longer be used. It is not as important to know what `day` of the month it is as to know what `dayofweek` or `dayoryear` it is. The `date` feature is also being dropped because it was only created for the holiday merge.

In [ ]:
# Drop day feat
meter.drop(['day', 'date'], axis=1, inplace=True)
meter.head(3)

In [ ]:
# Clean vars
del site_countries, US_holidays, CA_holidays, UK_holidays, IE_holidays, \
    US_holiday_df, CA_holiday_df, UK_holiday_df, IE_holiday_df, holiday_df
gc.collect()

## Section III: Data Processing

### Save dataframes

In [ ]:
# Create new subdir in data dir
output_dir = os.path.join('..', 'data', '03-feat-out')
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Save meter data
meter_path = os.path.join(output_dir, 'meter.pkl')
meter.to_pickle(meter_path)
pd.read_pickle(meter_path).info(null_counts=True)

In [ ]:
# Save weather data
weather_path = os.path.join(output_dir, 'weather.pkl')
weather.to_pickle(weather_path)
pd.read_pickle(weather_path).info()

In [ ]:
# Save building data
building_path = os.path.join(output_dir, 'building.pkl')
building.to_pickle(building_path)
pd.read_pickle(building_path).info()

### Merge all 3 dataframes into 1

In [ ]:
# Merge meter and building data
metr_bldg = pd.merge(meter.drop('site_id', axis=1), building, on='building_id', how='left')
metr_bldg.head(3)

In [ ]:
# Add weather data to the merged data
train = pd.merge(metr_bldg, weather, on=['site_id', 'timestamp'], how='left')
train.head(3)

In [ ]:
# Reorder cols
cols = ['site_id', 'building_id', 'meter', 'meter_type', 'timestamp', 'meter_reading', 
        'air_temperature', 'dew_temperature', 'rel_humidity', 'sea_level_pressure', 
        'wind_speed', 'wind_direction_x', 'wind_direction_y',  
        'primary_use', 'square_feet', 'year_built', 'missing_year', 'country', 
        'dayofyear', 'month', 'hour', 'dayofweek', 'is_weekend', 'is_holiday']
train = train[cols].copy()
train.info()

### Split train set by meter type

We saw in the EDA that different meter types had different usage patterns, so we will be splitting the dataset up into 4 parts (one for each meter type) and building a separate model for each meter type.

In [ ]:
# Split meter types into separate dfs
train_dfs = dict()
for m in ['electricity', 'chilledwater', 'steam', 'hotwater']:
    train_df = train[train['meter_type'] == m].drop(['meter', 'meter_type'], axis=1).copy()
    train_df.to_pickle(os.path.join(output_dir, m + '.pkl'))
    train_dfs[m] = train_df
    print(f'{m} meter: {train_df.shape}')

In [ ]:
# Clean vars
del meter, meter_path, weather, weather_path, building, building_path, metr_bldg, cols, m, train_df
gc.collect()

## Section IV: Feature Elimination - Electricity Meter

As more than half of the data are meter readings from electricity meters, we will be looking at electricity meters. The processed used here will be followed for the other 3 meter types.

Note: The letter "e" will be appended to variables to indicate that we are working with electricity meter data. For example, the variable data X will be called `Xe` and the target label y will be called `ye`.

### Data split - train, validation, test

The data is being split into 3 sets:
1. Training set - the machine learning model will be trained on this set
2. Validation set - model performance will be evaluated on this set during model tuning
3. Test set - this will be the holdout set for final testing once we have a finished model

We are splitting the data before the model selection process to minimize data leakage, that is, information from the validation or test set leaking into the training data in any way.

In [ ]:
# Split feats and target for electricity meter data
Xe = train_dfs['electricity'].drop('meter_reading', axis=1).copy()
ye = train_dfs['electricity']['meter_reading'].copy()
Xe.shape, ye.shape

In [ ]:
# Train/val/test split (70/12/18)
Xe_train, Xe_test, ye_train, ye_test = train_test_split(Xe, ye, test_size=0.3, random_state=30)
Xe_val, Xe_test, ye_val, ye_test = train_test_split(Xe_test, ye_test, test_size=0.6, random_state=30)

print('Train set:', Xe_train.shape, ye_train.shape)
print('Validation set:', Xe_val.shape, ye_val.shape)
print('Test set:', Xe_test.shape, ye_test.shape)

In [ ]:
# Clean vars
del Xe, ye
gc.collect()

### Rare label categorical encoding
The `primary_use` feature has a total of 16 categories, more than half of which would be considered "rare" in the data. That is, they occur less than a certain threshold (I will use the threshold 5%). This could be a problem for splitting the data because some rare labels may end up in the validation or test set, but not the training set. The model would not know what to do if it has never seen that label before.

To resolve this issue, we will group all these rare categories (any categories with a count less than 5% of the total data) into 1 label called "Rare".

In [ ]:
# Inspect primary use categories
udf.plot_rare_cats(Xe_train, 'primary_use')
plt.title('Counts of Primary Use Categories')
plt.ylabel('Number of meter readings')
plt.show()

In [ ]:
# Find common categories (has at least 5% of all values)
rarenc = RareLabelEncoder(tol=0.05, variables=['primary_use'])
rarenc.fit(Xe_train)
rarenc.encoder_dict_

In [ ]:
# Group rare categories together in all 3 sets
Xe_train = rarenc.transform(Xe_train)
Xe_val = rarenc.transform(Xe_val)
Xe_test = rarenc.transform(Xe_test)

Xe_train['primary_use'].value_counts()

### Mean target categorical encoding

The 2 categorical features (`primary_use` and `country`) still need to be encoded numerically. I will be using mean-target categorical encoding instead of one-hot encoding for 2 reasons:
1. To avoid expanding the feature set
2. The different categories showed differrent patterns in energy usage

In [ ]:
# Check the mean target of each primary use and country category
meanenc = MeanEncoder(variables=['primary_use', 'country'])
meanenc.fit(Xe_train, ye_train)
meanenc.encoder_dict_

In [ ]:
# Encode primary_use and country using the target mean in both train and validation sets
Xe_train = meanenc.transform(Xe_train)
Xe_val = meanenc.transform(Xe_val)
Xe_test = meanenc.transform(Xe_test)

Xe_train.head(3)

In [ ]:
# Dir paths for encoders
transformer_path = os.path.join('..', 'models', 'transformers')
rarenc_path = os.path.join(transformer_path, 'rare_enc')
meanenc_path = os.path.join(transformer_path, 'mean_enc')

# Create dirs
os.makedirs(rarenc_path, exist_ok=True)
os.makedirs(meanenc_path, exist_ok=True)

# Save encoders
joblib.dump(rarenc, os.path.join(rarenc_path, 'rare_enc0.pkl'))
joblib.dump(meanenc, os.path.join(meanenc_path, 'mean_enc0.pkl'))

### Feature elimination

To start off the feature selection process, we will be looking for the following redundant features to remove:
1. Duplicated features - pairs of features that share the same value for all observations
2. Constant features - features that only have 1 unique value
3. Quasi-constant features - features that have 1 unique value for MOST of their observations (low variance)
    - I will be using a variance threshold of 1%
4. Correlated features - pairs of features with high linear correlation
    - I will be using a correlation threshold of 0.5

In [ ]:
# Duplicated feats
udf.duplicated_feats(Xe_train)

In [ ]:
# Constant and quasi-constant feats
udf.constant_feats(Xe_train)

In [ ]:
# Correlated feats
udf.correlated_feats(Xe_train.corr(), threshold=0.5)

There are no duplicated, constant, or even quasi-constant features. However, there are 4 pairs of correlated features here, all of which have a high correlation constant (much higher than the 0.5 threshold). Let's take a closer look at the feature correlation.

### Correlated features

In [ ]:
# Feat correlation
corr = Xe_train.corr()
corr.style.applymap(lambda c: 'color: red; background-color: yellow' if 0.5 < abs(c) < 1 else '')

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap=sns.color_palette('Reds', 8))
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Drop timestamp and 1 of 2 feats from each correlated pair in all 3 sets
to_drop = ['timestamp', 'building_id', 'dew_temperature', 'month', 'dayofweek']
Xe_train.drop(to_drop, axis=1, inplace=True)
Xe_val.drop(to_drop, axis=1, inplace=True)
Xe_test.drop(to_drop, axis=1, inplace=True)

Xe_train.shape, Xe_val.shape, Xe_test.shape

In [ ]:
# Clean vars
del rarenc_path, meanenc_path, corr, to_drop
gc.collect()

For each of the 4 pairs of corrrelated features, the feature that was dropped is either harder to deal with, harder to understand, or just not as well-suited as the other feature. Features dropped:
- `Building_id` - has much higher cardinality than `site_id`, which makes `site_id` easier to encode
    - Even though `site_id` and `building_id` are numeric, there is no order to the values because these are categorical variables
- `Dew_temperature` - tracks `air_temperature` closely, but is not as concrete conceptually as `air_temperature`
- `Month` - not as fine-tuned as `dayofyear` values, so the model may have a hard time differentiating different days of a month
- `Dayofweek` - not as important to know what day of the week it is as it is to just know whether that day was a weekday or weekend

## Section V: Feature Selection - Electricity Meter

After eliminating redundant features, there are 16 remaining. We will be looking at several feature selection methods to get an idea of which of the 16 features will be the most important in predicting energy consumption.

### Feature scaling

The remaining features are on completely different scales, with boolean features consisting of either 0 or 1 and a feature like `square_feet` containing values in the order of 10 to the 6th power. The values need to be on the same scale so that some of the models (especially linear models) do not misinterpret the data. We will be normalizing the values of every feature with the standard scaling method, where each feature will have a mean of 0 and a standard deviation of 1.

In [ ]:
# Scale all feats using the mean and stdev
scaler = StandardScaler()
scaler.fit(Xe_train)

# Apply the scaler to all 3 sets
Xe_train_scaled = pd.DataFrame(scaler.transform(Xe_train), columns=Xe_train.columns)
Xe_val_scaled = pd.DataFrame(scaler.transform(Xe_val), columns=Xe_val.columns)
Xe_test_scaled = pd.DataFrame(scaler.transform(Xe_test), columns=Xe_test.columns)

Xe_train_scaled.head(3)

In [ ]:
# Create dir for scaler
scaler_path = os.path.join(transformer_path, 'scaler')
os.makedirs(scaler_path, exist_ok=True)

# Save scaler
joblib.dump(scaler, os.path.join(scaler_path, 'scaler0.pkl'))

### Track how many times each feature is selected

Since we are looking at several methods of selecting features, let's create a table to track the running total of how many times each feature is selected. The idea is that a feature would be a better predictor the more it is selected, and any feature with a count of 0 is probably not contributing very much in the prediction.

In [ ]:
# Df to track selection count
feat_sel = pd.DataFrame([0] * Xe_train.shape[1], index=Xe_train.columns, columns=['count'])
feat_sel

### Correlation with target

To start off, let's check the linear correlation of each feature with the target label.

In [ ]:
# Merge target var into the feats
e_train = pd.merge(Xe_train, ye_train, how='left', left_index=True, right_index=True)

# Feat correlation with target var
e_corr = e_train.corr()
e_corr['meter_reading'].sort_values(ascending=False)

In [ ]:
# Plot correlation
e_corr['meter_reading'].sort_values()[:-1].plot.barh(figsize=(15, 5), title='Feature Correlation with Target Variable')
plt.show()

Most of the features have a really low correlation with the target. It looks like `square_feet` and `country` would be the best predictors if we were to use a linear model.

In [ ]:
# Add feats with a correlation coef magnitude of at least 0.4
corr_feats = e_corr[e_corr['meter_reading'].abs() >= 0.4].index[:-1].tolist()
feat_sel = udf.inc_feat_count(feat_sel, corr_feats)
feat_sel

In [ ]:
# Clean vars
del transformer_path, scaler_path, e_train, e_corr
gc.collect()

### Lasso regularization

Lasso (L1) regularization is able to reduce feature coefficients to 0. This removes the less informative features from the model. Using this selection method, the remaining features are the ones that are "selected". We will be doing this 2 different ways:
1. Select from model - select features with high coefficients
2. Recursive feature eliminaton - recursively remove the feature with the lowest coefficient until half of the features remain

In [ ]:
# Select feats using L1 regularization
l1_feats = udf.feats_from_model(Xe_train_scaled, ye_train, SelectFromModel, Lasso(alpha=10))
l1_feats

In [ ]:
# Select feats recursively using L1 regularization
rfe_l1_feats = udf.feats_from_model(Xe_train_scaled, ye_train, RFE, Lasso(alpha=10))
rfe_l1_feats

In [ ]:
# Increment the counts for selected feats
feat_sel = udf.inc_feat_count(feat_sel, l1_feats + rfe_l1_feats)
feat_sel

Aside from the 3 features that have the highest correlation with the target - `square_feet`, `country`, and `primary_use` - the day and time are also providing a lot of information.

### Tree importance

We'll be repeating the above process with a decision tree model. As decision tree is a nonlinear model, it may provide completely different insights into which features are important in predicting energy consumption.

In [ ]:
# Select feats using decision tree
tree_feats = udf.feats_from_model(Xe_train_scaled, ye_train, SelectFromModel, DecisionTreeRegressor())
tree_feats

In [ ]:
# Select feats recursively using decision tree
rfe_tree_feats = udf.feats_from_model(Xe_train_scaled, ye_train, RFE, DecisionTreeRegressor())
rfe_tree_feats

In [ ]:
# Increment the counts for selected feats
feat_sel = udf.inc_feat_count(feat_sel, tree_feats + rfe_tree_feats)
feat_sel

Again, the 3 features most correlated with the target show up, as well as day and time. But interestingly, we're also seeing `year_built` and `air_temperature` here.

## Section VI: Featurization Conclusion

### Create dataframe of selected features for each method

In [ ]:
# Method names and their selected methods
sel_feats = {
    'target_corr': corr_feats,
    'lasso_coef':  l1_feats, 
    'lasso_coef_recursive': rfe_l1_feats, 
    'tree_importance': tree_feats, 
    'tree_importance_recursive': rfe_tree_feats
}

# Fill shorter lists to match the length of the longest list
longest = max([len(lst) for lst in sel_feats.values()])
for key, val in sel_feats.items():
    sel_feats[key] += [''] * (longest - len(val))
    
# Selected feats for each method
sel_feats_df = pd.DataFrame(sel_feats).fillna('')
sel_feats_df

In [ ]:
# Clean vars
del corr_feats, l1_feats, rfe_l1_feats, tree_feats, rfe_tree_feats, sel_feats, longest, key, val
gc.collect()

There is strong evidence that `square_feet` and `country` will be the best predictors of energy consumption. Besides this, `primary_use` and day/time features will likely be good predictors as well. The following features were only selected once but may be informative nonetheless:
- `site_id`
- `air_temperature`
- `missing_year`
- `is_holiday`

The 5 features that weren't selected at all didn't do well in both linear and tree-based models. They may end up being useful, but for now, these are the prime candidates to drop from the feature set:
- `rel_humidity`
- `sea_level_pressure`
- `wind_direction_x`
- `wind_direction_y`
- `wind_speed`

With this, we have a pretty good idea of which features will and will not be useful in predicting energy consumption. We won't know for sure until actually building the models, but this will serve as a good starting point for making the feature set.

### Save data

In [ ]:
feats_path = os.path.join(output_dir, 'feats.pkl')
sel_feats_df.to_pickle(feats_path)
pd.read_pickle(feats_path)